In [1]:
import duckdb
import pandas as pd

con = duckdb.connect("../db/ecommerce.duckdb")


## Analysis 1 – Overall orders and revenue

**Business question:**  
What is the total number of orders in the system, and what is the total revenue generated?

This gives a quick high-level view of how big the dataset is and how much money our synthetic e-commerce store has made.


In [2]:
q1 = """
SELECT 
    COUNT(*) AS total_orders,
    SUM(total_amount) AS total_revenue
FROM orders;
"""

df1 = con.execute(q1).df()
df1


,total_orders,total_revenue
0,100000,1.438998e+08


## Analysis 2 – Revenue by customer country

**Business question:**  
Which customer countries generate the most revenue?

This helps us understand which markets are most valuable and where our users are spending more.


In [3]:
q2 = """
SELECT 
    u.country,
    COUNT(o.order_id) AS num_orders,
    SUM(o.total_amount) AS total_revenue,
    AVG(o.total_amount) AS avg_order_value
FROM orders o
JOIN users u 
    ON o.user_id = u.user_id
GROUP BY u.country
ORDER BY total_revenue DESC;
"""

df2 = con.execute(q2).df()
df2


,country,num_orders,total_revenue,avg_order_value
0,USA,17139,24717936.00,1442.204096
1,India,16782,24234251.58,1444.062184
2,UK,16847,24172679.43,1434.835842
3,Germany,16792,24131703.21,1437.095236
4,Australia,16236,23334606.22,1437.213983
5,Canada,16204,23308661.60,1438.451098


## Analysis 3 – Monthly revenue trend

**Business question:**  
How has revenue changed over time month by month?

This helps us see growth, seasonality, or drops in overall sales.


In [4]:
q3 = """
SELECT 
    DATE_TRUNC('month', order_date) AS month,
    COUNT(*) AS num_orders,
    SUM(total_amount) AS total_revenue,
    AVG(total_amount) AS avg_order_value
FROM orders
GROUP BY month
ORDER BY month;
"""

df3 = con.execute(q3).df()
df3.head()


,month,num_orders,total_revenue,avg_order_value
0,2022-01-01,2777,4011721.05,1444.624073
1,2022-02-01,2618,3776037.98,1442.336891
2,2022-03-01,2882,4173533.17,1448.137811
3,2022-04-01,2810,3978891.85,1415.975747
4,2022-05-01,2757,3989151.54,1446.917497


## Analysis 4 – Week-over-week revenue growth

**Business question:**  
How does revenue change week over week? Are there weeks with strong growth or decline?

This uses a window function (`LAG`) to compare each week's revenue with the previous week.


In [5]:
q4 = """
WITH weekly AS (
    SELECT 
        DATE_TRUNC('week', order_date) AS week_start,
        SUM(total_amount) AS revenue
    FROM orders
    GROUP BY week_start
)
SELECT 
    week_start,
    revenue,
    LAG(revenue) OVER (ORDER BY week_start) AS prev_week_revenue,
    (revenue - LAG(revenue) OVER (ORDER BY week_start)) AS wow_change,
    CASE 
        WHEN LAG(revenue) OVER (ORDER BY week_start) IS NULL THEN NULL
        ELSE (revenue - LAG(revenue) OVER (ORDER BY week_start))
             / LAG(revenue) OVER (ORDER BY week_start) * 100
    END AS wow_pct_change
FROM weekly
ORDER BY week_start;
"""

df4 = con.execute(q4).df()
df4.head()


,week_start,revenue,prev_week_revenue,wow_change,wow_pct_change
0,2021-12-27,261231.06,NaN,NaN,NaN
1,2022-01-03,839369.97,261231.06,578138.91,221.313235
2,2022-01-10,910029.75,839369.97,70659.78,8.418193
3,2022-01-17,924382.01,910029.75,14352.26,1.577120
4,2022-01-24,918530.90,924382.01,-5851.11,-0.632975


## Analysis 5 – Top 10 customers by total spend

**Business question:**  
Who are our highest-value customers based on total money spent?

This helps identify VIP users for loyalty programs, discounts, or targeted marketing.


In [6]:
q5 = """
SELECT 
    u.user_id,
    u.first_name,
    u.last_name,
    SUM(o.total_amount) AS total_spent,
    COUNT(o.order_id) AS num_orders
FROM orders o
JOIN users u 
    ON o.user_id = u.user_id
GROUP BY u.user_id, u.first_name, u.last_name
ORDER BY total_spent DESC
LIMIT 10;
"""

df5 = con.execute(q5).df()
df5


,user_id,first_name,last_name,total_spent,num_orders
0,21599,Thomas,Perry,19249.40,9
1,9700,Natalie,Bell,17789.83,8
2,26672,Michael,Mccoy,16761.30,7
3,40376,Jacob,Davis,16574.63,6
4,5785,Jacob,Bush,16565.58,8
5,45590,Rebecca,Everett,16149.87,7
6,48879,Joel,Holden,15837.35,6
7,46003,Rodney,Thomas,15744.19,7
8,28106,John,Hicks,15565.41,7
9,33479,Jennifer,Johnson,15265.04,7


## Analysis 6 – Customer value segments

**Business question:**  
How can we segment customers into 'High value', 'Medium value', and 'Low value' based on their total spend?

This helps us think about different strategies for different customer groups.


In [7]:
q6 = """
WITH user_spend AS (
    SELECT 
        u.user_id,
        u.first_name,
        u.last_name,
        COALESCE(SUM(o.total_amount), 0) AS total_spent
    FROM users u
    LEFT JOIN orders o 
        ON u.user_id = o.user_id
    GROUP BY u.user_id, u.first_name, u.last_name
)
SELECT 
    user_id,
    first_name,
    last_name,
    total_spent,
    CASE
        WHEN total_spent = 0 THEN 'No spend'
        WHEN total_spent < 100 THEN 'Low value'
        WHEN total_spent < 500 THEN 'Medium value'
        ELSE 'High value'
    END AS segment
FROM user_spend
ORDER BY total_spent DESC NULLS LAST
LIMIT 30;
"""

df6 = con.execute(q6).df()
df6


,user_id,first_name,last_name,total_spent,segment
0,21599,Thomas,Perry,19249.40,High value
1,9700,Natalie,Bell,17789.83,High value
2,26672,Michael,Mccoy,16761.30,High value
3,40376,Jacob,Davis,16574.63,High value
4,5785,Jacob,Bush,16565.58,High value
5,45590,Rebecca,Everett,16149.87,High value
6,48879,Joel,Holden,15837.35,High value
7,46003,Rodney,Thomas,15744.19,High value
8,28106,John,Hicks,15565.41,High value
9,33479,Jennifer,Johnson,15265.04,High value


## Analysis 7 – Repeat purchase rate

**Business question:**  
What percentage of customers placed more than one order?

This tells us how many users come back and buy again (basic retention metric).


In [8]:
q7 = """
WITH user_order_counts AS (
    SELECT 
        user_id,
        COUNT(*) AS num_orders
    FROM orders
    GROUP BY user_id
)
SELECT
    SUM(CASE WHEN num_orders >= 2 THEN 1 ELSE 0 END) * 100.0
    / COUNT(*) AS repeat_purchase_rate_pct
FROM user_order_counts;
"""

df7 = con.execute(q7).df()
df7


,repeat_purchase_rate_pct
0,68.903301
